In [13]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [5]:
def create_conn_dwh():
    connection = psycopg2.connect(
        dbname='postgres',
        user='ste',
        password='ILzAYQ72aEe9',
        host='primarydwhcsd.aerxd.tech',
        port=6432
    )
    return connection

def create_connection():
    return psycopg2.connect(
        dbname='csd_bi',
        user='datalens_utl',
        password='mQnXQaHP6zkOaFdTLRVLx40gT4',
        host='138.68.88.175',
        port=5432
    )

def run_query_with_conn(conn, query):
    with conn.cursor() as cursor:
        cursor.execute(query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(rows, columns=columns)
    return df

In [6]:
query_get_data = """
    select
        extra -> 'cardDetails' ->> 'pan' as pan,
        i.created_at,
        COUNT(i.id) as orders_count
    from orders.invoice i
    JOIN lists.invoice_order_type_list iotl ON i.order_type_id = iotl.id
    where i.created_at >= '2025-08-01' and iotl.system_name ILIKE '%H2H%'
    group by
        extra -> 'cardDetails' ->> 'pan',
        i.created_at
    order by COUNT(i.id) desc
"""


In [9]:
connection = create_connection()


In [12]:
data = run_query_with_conn(connection, query_get_data)

In [14]:
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/postgres'.format(
        user='ste',
        password='ILzAYQ72aEe9',
        host='primarydwhcsd.aerxd.tech',
        port=6432)
    )

In [16]:
data.to_sql(
            schema='cascade',
            name='cards_info_sg',
            if_exists='append',
            con=engine,
            index=False
        )

644